In [1]:
import pandas as pd
import json
import os
from parrot import Parrot
from tqdm import tqdm

In [5]:
hc3_df = pd.read_json(path_or_buf="../../datasets/reddit_eli5.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN
...,...,...,...,...
17107,Why is the general Xbox One reveal feedback ne...,[A few items people do n't seem to like : 1 . ...,[It's difficult to say exactly why the general...,NaN
17108,If infrared is thermal why does n't my remote ...,"[It 's very , very low power . Around the orde...",[Infrared light is a type of light that we can...,NaN
17109,Why are cars all about their brake - horse pow...,[Brake horsepower has nothing to do with brake...,"[Cars are not just about how fast they can go,...",NaN
17110,"What does a CO^2 scrubber do , and how does it...",[A [ scrubber ] ( URL_0 ) is a engineering pro...,[A CO2 scrubber is a device that removes carbo...,NaN


In [3]:
parrot = Parrot()

In [6]:
# add a new column to the df
hc3_df["paraphrased_parrot"] = None

n = len(hc3_df['chatgpt_answers'])

for i in tqdm(range(n),
               desc="Loading…",
               ascii=False):
    list_of_chatgpt_ans = hc3_df.at[i, 'chatgpt_answers']
    list_of_paraphrased_responses = []

    if len(list_of_chatgpt_ans)<=0:
        hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
        continue
    
    for chatgpt_ans in list_of_chatgpt_ans:
        para_phrases = parrot.augment(input_phrase=chatgpt_ans,
                                      use_gpu=True,
                                      max_return_phrases = 5, 
                                      max_length=int(len(chatgpt_ans)*2)
                                      )
        if para_phrases:
            list_of_paraphrased_responses.append(para_phrases[0][0])
    hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
hc3_df.head()

Loading…: 100%|██████████████████████████████████████████████████████████████| 17112/17112 [10:06:21<00:00,  2.13s/it]


,question,human_answers,chatgpt_answers,index,paraphrased_parrot
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,[there are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,[salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,[it is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,[when the wright brothers made the first power...


In [7]:
hc3_df.to_csv("../../datasets/parrot_augmented_eli5.csv", index=False)

In [9]:
# run the paraphrasing tool again to fill in cells that are empty during the first round.
for i in tqdm(range(n),
               desc="Loading…",
               ascii=False):
    list_of_chatgpt_ans = hc3_df.at[i, 'chatgpt_answers']
    list_of_paraphrased_responses = []

    if len(list_of_chatgpt_ans)<=0:
        hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
        continue
        
    if len(hc3_df.at[i, "paraphrased_parrot"])<1:
        for chatgpt_ans in list_of_chatgpt_ans:
            para_phrases = parrot.augment(input_phrase=chatgpt_ans,
                                          use_gpu=True,
                                          max_return_phrases = 10,
                                          max_length=int(len(chatgpt_ans)*2.5)
                                          )
            if para_phrases:
                list_of_paraphrased_responses.append(para_phrases[0][0])
        hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
    else:
        continue
    
    
hc3_df.head()

Loading…: 100%|██████████████████████████████████████████████████| 17112/17112 [26:17<00:00, 10.85it/s]


,question,human_answers,chatgpt_answers,index,paraphrased_parrot
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,[there are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,[salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,[it is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,[when the wright brothers made the first power...


In [15]:
# fix the faulty paraphrase from the previous cell
for i in tqdm(range(n),
               desc="Loading…",
               ascii=False):
    list_of_chatgpt_ans = hc3_df.at[i, 'chatgpt_answers']
    list_of_paraphrased_responses = []
        
    if i==1663:
        for chatgpt_ans in list_of_chatgpt_ans:
            para_phrases = parrot.augment(input_phrase=chatgpt_ans,
                                          use_gpu=True,
                                          max_return_phrases = 10,
                                          max_length=int(len(chatgpt_ans)*2.5)
                                          )
            if para_phrases:
                list_of_paraphrased_responses.append(para_phrases[0][0])
        hc3_df.at[i, 'paraphrased_parrot'] = list_of_paraphrased_responses
    else:
        continue
    
    
hc3_df.head()

Loading…: 100%|████████████████████████████████████████████████| 17112/17112 [00:02<00:00, 8136.80it/s]


,question,human_answers,chatgpt_answers,index,paraphrased_parrot
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,[there are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,[salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,[it is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,[when the wright brothers made the first power...


In [17]:
hc3_df.at[1663, "chatgpt_answers"]

["Becoming the President of the United States is a very big and important job, and it's not easy to do. Here's how you can become President: You have to be at least 35 years old and be a natural-born citizen of the United States. You have to live in the United States for at least 14 years. You have to be qualified to hold office according to the Constitution of the United States. You have to run for office and win an election. To do this, you have to campaign and get a lot of people to vote for you. After you win an election, you become the President of the United States. It takes a lot of hard work and determination to become President, and it's not something that happens overnight. But if you're serious about it and you're willing to put in the effort, it's possible to achieve your goal."]

In [16]:
hc3_df.at[1663, "paraphrased_parrot"]

["it is a big and important job and it's not easy to become president but if you are serious about it and you're willing to put in the effort it's possible to achieve your goal"]

In [18]:
hc3_df.to_csv("../../datasets/parrot_augmented_eli5_v2.csv", index=False)